# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/wXfah/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298572975118                   -0.85    5.1         
  2   -8.300220627766       -2.78       -1.26    1.1   83.2ms
  3   -8.300438699203       -3.66       -1.89    2.2   97.4ms
  4   -8.300459885776       -4.67       -2.72    2.4    100ms
  5   -8.300463542766       -5.44       -3.05    2.5    110ms
  6   -8.300464054412       -6.29       -3.20   12.1    226ms
  7   -8.300464346659       -6.53       -3.34    1.6   88.4ms
  8   -8.300464488029       -6.85       -3.47    2.1   90.7ms
  9   -8.300464619759       -6.88       -3.75    1.6   91.5ms
 10   -8.300464635051       -7.82       -3.91    1.4   93.2ms
 11   -8.300464640561       -8.26       -4.17    1.5    108ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67384478381                   -0.70    6.1         
  2   -16.67787813532       -2.39       -1.14    1.0    191ms
  3   -16.67919778098       -2.88       -1.87    3.4    254ms
  4   -16.67927387029       -4.12       -2.70    3.1    252ms
  5   -16.67928499689       -4.95       -3.04    4.9    302ms
  6   -16.67928619350       -5.92       -3.44    1.9    202ms
  7   -16.67928621570       -7.65       -3.95    2.2    218ms
  8   -16.67928622147       -8.24       -4.55    3.5    251ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32553792011                   -0.56    8.0         
  2   -33.33275550525       -2.14       -1.00    1.2    656ms
  3   -33.33407071377       -2.88       -1.73    3.9    836ms
  4   -33.33579659032       -2.76       -2.24    2.8    767ms
  5   -33.33689875174       -2.96       -2.46   11.8    1.24s
  6   -33.33692986089       -4.51       -2.74    3.0    722ms
  7   -33.33694366500       -4.86       -3.66    2.8    792ms
  8   -33.33694370084       -7.45       -3.78    4.9    914ms
  9   -33.33694376364       -7.20       -4.18    1.6    626ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298481175512                   -0.85    5.2         
  2   -8.300273415098       -2.75       -1.59    1.0   62.7ms
  3   -8.300425803925       -3.82       -2.50    2.6   98.5ms
  4   -8.300353024782   +   -4.14       -2.23    4.2    120ms
  5   -8.300464079562       -3.95       -3.46    1.1   71.2ms
  6   -8.300464550461       -6.33       -3.74    3.8    124ms
  7   -8.300464637715       -7.06       -4.24    1.1   75.4ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32540434502                   -0.56    6.6         
  2   -33.32838326019       -2.53       -1.27    1.2    583ms
  3   -23.97373403689   +    0.97       -0.58    5.9    1.11s
  4   -33.24358240675        0.97       -1.54    5.2    1.06s
  5   -33.15191770664   +   -1.04       -1.28    3.1    849ms
  6   -32.04536045450   +    0.04       -1.01    5.2    975ms
  7   -33.32645864773        0.11       -1.91    4.5    874ms
  8   -33.33306079925       -2.18       -2.21    2.1    657ms
  9   -33.33550709191       -2.61       -2.35    2.1    698ms
 10   -33.33506919073   +   -3.36       -2.40    1.6    643ms
 11   -33.33620117589       -2.95       -2.58    1.6    616ms
 12   -33.33672491735       -3.28      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.